In [ ]:
import pandas as pd

df = pd.read_json('data\General_validation_text.jsonl', lines = True)
df.drop(columns = ['id'], inplace = True)
df.rename(columns = {'article' : 'input_text', 'highlights': "target_text"}, inplace = True)
df.to_json('data\General_validation_text.jsonl', orient = 'records',lines = True)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import Dataset
import torch

#load long-T5
model_name = "google/long-t5-tglobal-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data = pd.read_json('data/General_train_text_chunk_1.jsonl', lines = True)
dataset = Dataset.from_dict(data)

#Tokenize
def preprocess(example):
    input_text = ["summarize: " + x for x in example["input_text"]]
    inputs = tokenizer(input_text, padding = "max_length", max_length = 4096, truncation = True)
    targets = tokenizer(example["target_text"], padding = "max_length", max_length = 256, truncation = True)

    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched = True, remove_columns = dataset.column_names)

training_args = TrainingArguments(
    output_dir="./longt5-summarizer",            
    per_device_train_batch_size=1,
    num_train_epochs=3,
    save_strategy="epoch",               
    logging_dir="./logs",                
    fp16=torch.cuda.is_available(),      
    save_total_limit=2
)

#Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    tokenizer = tokenizer
)

In [ ]:
trainer.train()

In [ ]:
import pandas as pd

df = pd.read_json('data\General_train_text_chunk_1.jsonl', lines = True)
# your_article_text = df.at[0, 'input_text']
# ans = df.at[0, 'target_text']

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/long-t5-tglobal-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
text = "summarize: " + your_article_text
inputs = tokenizer(text, return_tensors="pt", max_length=4096, truncation=True)

summary_ids = model.generate(inputs["input_ids"], max_length=256, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)
print(ans)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Choose your model (e.g., "t5-small" or "google/long-t5-tglobal-base")
model_name = "t5-small"  # or your actual model name

# Load model and tokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Save them locally
tokenizer.save_pretrained("./my_local_t5/tokenizer")
model.save_pretrained("./my_local_t5/model")


In [ ]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("audio.mp3")
print(result["text"])